In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

import os
import sys

project_path = os.path.join(os.getcwd(), '..','..')
sys.path.append(project_path)

from utils.transformation import reusable

#### **DimUsers**

#### **AUTOLOADER**

In [0]:
df_user = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format", "parquet")\
        .option("cloudFiles.schemaLocation", 'abfss://silver@storageazurepro.dfs.core.windows.net/DimUser/checkpoint')\
        .option('schemaEvolutionMode', 'addNewColumns')\
        .load("abfss://bronze@storageazurepro.dfs.core.windows.net/DimUser")

In [0]:
df_user = reusable().dropColumns(df_user, ['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])

In [0]:
df_user.writeStream.format("delta")\
        .outputMode('append')\
        .option("checkpointLocation",'abfss://silver@storageazurepro.dfs.core.windows.net/DimUser/checkpoint')\
        .trigger(once=True)\
        .option('path', 'abfss://silver@storageazurepro.dfs.core.windows.net/DimUser/data')\
        .toTable('spotify_cata.silver.DimUser')

In [0]:
%sql
SELECT *
FROM spotify_cata.silver.dimuser

In [0]:
df = spark.read.table('spotify_cata.silver.DimUser')
display(df)

#### **DimArtist**

In [0]:
df_artist = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format", "parquet")\
        .option("cloudFiles.schemaLocation", 'abfss://silver@storageazurepro.dfs.core.windows.net/DimArtist/checkpoint')\
        .option('schemaEvolutionMode', 'addNewColumns')\
        .load("abfss://bronze@storageazurepro.dfs.core.windows.net/DimArtist")

In [0]:
df_artist_obj = reusable()

df_artist = df_artist_obj.dropColumns(df_artist, ['_rescued_data'])
df_artist = df_artist.dropDuplicates(['artist_id'])

In [0]:
df_artist.writeStream.format("delta")\
        .outputMode('append')\
        .option("checkpointLocation",'abfss://silver@storageazurepro.dfs.core.windows.net/DimArtist/checkpoint')\
        .trigger(once=True)\
        .option('path', 'abfss://silver@storageazurepro.dfs.core.windows.net/DimArtist/data')\
        .toTable('spotify_cata.silver.DimArtist')

#### **DimTrack**

In [0]:
df_track = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format", "parquet")\
        .option("cloudFiles.schemaLocation", 'abfss://silver@storageazurepro.dfs.core.windows.net/DimTrack/checkpoint')\
        .option('schemaEvolutionMode', 'addNewColumns')\
        .load("abfss://bronze@storageazurepro.dfs.core.windows.net/DimTrack")

In [0]:
df_track = df_track.withColumn('durationFlag', when(col("duration_sec") < 150, 'low')\
                                            .when(col("duration_sec") < 300, 'medium')\
                                            .otherwise('high'))
df_track = df_track.withColumn('track_name', regexp_replace(col('track_name'), '-', ' '))
df_track = reusable().dropColumns(df_track, ['_rescued_data'])

In [0]:
df_track.writeStream.format("delta")\
        .outputMode('append')\
        .option("checkpointLocation",'abfss://silver@storageazurepro.dfs.core.windows.net/DimTrack/checkpoint')\
        .trigger(once=True)\
        .option('path', 'abfss://silver@storageazurepro.dfs.core.windows.net/DimTrack/data')\
        .toTable('spotify_cata.silver.DimTrack')

#### **DimDate**

In [0]:
df_date = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format", "parquet")\
        .option("cloudFiles.schemaLocation", 'abfss://silver@storageazurepro.dfs.core.windows.net/DimDate/checkpoint')\
        .option('schemaEvolutionMode', 'addNewColumns')\
        .load("abfss://bronze@storageazurepro.dfs.core.windows.net/DimDate")

In [0]:
df_date = reusable().dropColumns(df_date, ['_rescued_data'])

df_date.writeStream.format("delta")\
        .outputMode('append')\
        .option("checkpointLocation",'abfss://silver@storageazurepro.dfs.core.windows.net/DimDate/checkpoint')\
        .trigger(once=True)\
        .option('path', 'abfss://silver@storageazurepro.dfs.core.windows.net/DimDate/data')\
        .toTable('spotify_cata.silver.DimDate')

#### **FactStream**

In [0]:
df_fact = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format", "parquet")\
        .option("cloudFiles.schemaLocation", 'abfss://silver@storageazurepro.dfs.core.windows.net/FactStream/checkpoint')\
        .option('schemaEvolutionMode', 'addNewColumns')\
        .load("abfss://bronze@storageazurepro.dfs.core.windows.net/FactStream")

In [0]:
df_fact = reusable().dropColumns(df_fact, ['_rescued_data'])

df_fact.writeStream.format("delta")\
        .outputMode('append')\
        .option("checkpointLocation",'abfss://silver@storageazurepro.dfs.core.windows.net/FactStream/checkpoint')\
        .trigger(once=True)\
        .option('path', 'abfss://silver@storageazurepro.dfs.core.windows.net/FactStream/data')\
        .toTable('spotify_cata.silver.FactStream')

In [0]:
%sql
SELECT *
FROM spotify_cata.gold.dimtrack
WHERE track_id IN (46,5)